### **Cargamos el conjunto de datos**

In [ ]:
import pandas as pd

df = pd.read_csv('dset1.csv')

# df.head()
df.describe()

### **Seleccionamos las columnas a utilizar y nos aseguramos de usar dropna por si existen valores nulos**

In [ ]:
df_data = df[['Duracion_Estimada_Meses', 'Tamaño_Equipo', 'Tecnologias_Usadas',
              'Experiencia_Equipo_Años', 'Presupuesto_Cliente_USD',
              'Cobro_Proyecto_USD']]

df_data = df_data.dropna()

### **Categorizamos las tecnologías en columnas**

In [ ]:
from sklearn.preprocessing import MultiLabelBinarizer

# Convertir las tecnologías en una lista
df_data['Tecnologias_Usadas'] = df_data['Tecnologias_Usadas'].apply(lambda x: x.split(', '))

mlb = MultiLabelBinarizer()

# Aplicar el MultiLabelBinarizer para convertir las listas de tecnologías en columnas binarias
tecnologias_encoded = pd.DataFrame(mlb.fit_transform(df_data['Tecnologias_Usadas']),
                                   columns=mlb.classes_,
                                   index=df_data.index)

df_data = df_data.drop('Tecnologias_Usadas', axis=1)
df_data = pd.concat([df_data, tecnologias_encoded], axis=1)

df_data.head()

### **Categorizamos los niveles**

In [ ]:
categoras = df[['Nivel_Complejidad', 'Tipo_Proyecto']]

cat_num = pd.get_dummies(categoras, drop_first=True)
cat_num.head()

df_data_train = pd.concat([df_data, cat_num], axis=1)
df_data_train.head()

### **Gráficos para análisis**

In [ ]:
import seaborn

seaborn.pairplot(df_data.drop('Cobro_Proyecto_USD', axis = 1))

In [ ]:
seaborn.heatmap(df_data.corr(),
                xticklabels=df_data.columns,
                yticklabels=df_data.columns)

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 6))
plt.bar(df_data['Duracion_Estimada_Meses'], df_data['Cobro_Proyecto_USD'])
plt.xlabel('Duración Estimada (Meses)')
plt.ylabel('Cobro del Proyecto (USD)')
plt.title('Relación entre Duración Estimada y Cobro del Proyecto')
plt.show()

In [ ]:
import matplotlib.pyplot as plt

# Agrupar los datos por tipo de proyecto y calcular la media del cobro
promedio_cobro_por_tipo = df.groupby('Tipo_Proyecto')['Cobro_Proyecto_USD'].mean()

plt.figure(figsize=(10, 6))
plt.bar(promedio_cobro_por_tipo.index, promedio_cobro_por_tipo.values)
plt.xlabel('Tipo de Proyecto')
plt.ylabel('Cobro Promedio (USD)')
plt.title('Cobro Promedio por Tipo de Proyecto')
plt.xticks(promedio_cobro_por_tipo.index, rotation=45, ha='right')
plt.tight_layout()
plt.show()

### **Proceso de entrenamiento**

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Dividir las características (X) y la variable objetivo (y)
x = df_data_train.drop('Cobro_Proyecto_USD', axis=1)
y = df_data_train['Cobro_Proyecto_USD']

# Dividir los datos en conjunto de entrenamiento y prueba (80% entrenamiento, 20% prueba)
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=0)

sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_absolute_error, r2_score, root_mean_squared_error

# 1 Modelo de Regresión Lineal
linear_model = LinearRegression()

# Entrenar el modelo
linear_model.fit(X_train, y_train)

# Predecir en el conjunto de prueba
y_pred_linear = linear_model.predict(X_test)

# Evaluar el modelo
print("Regresión Lineal:")
print("MSE:", round(root_mean_squared_error(y_test, y_pred_linear), 2))
print("MAE:", round(mean_absolute_error(y_test, y_pred_linear), 2))
print("R2 Score:", round(r2_score(y_test, y_pred_linear), 2))

# 2 Modelo de Random Forest
forest_model = RandomForestRegressor()

# Entrenar el modelo
res_fit = forest_model.fit(X_train, y_train)

# Predecir en el conjunto de prueba
y_pred_forest = forest_model.predict(X_test)

# Evaluar el modelo
print("\nRandom Forest:")
print("MSE:", round(root_mean_squared_error(y_test, y_pred_forest), 2))
print("MAE:", round(mean_absolute_error(y_test, y_pred_forest), 2))
print("R2 Score:", round(r2_score(y_test, y_pred_forest), 2))

# 3 Modelo GradientBoosting
gradient_boosting_model = GradientBoostingRegressor()
gradient_boosting_model.fit(X_train, y_train)

y_pred_gradient_boosting = gradient_boosting_model.predict(X_test)

print("\nGradient Boosting:")
print("MSE:", round(root_mean_squared_error(y_test, y_pred_gradient_boosting), 2))
print("MAE:", round(mean_absolute_error(y_test, y_pred_gradient_boosting), 2))
print("R2 Score:", round(r2_score(y_test, y_pred_gradient_boosting), 2))

### **Entrenamiento de prueba con TensorFlow**

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

model = keras.Sequential([
    keras.Input(shape=(X_train.shape[1],)),
    layers.Dense(256, activation='relu'),
    layers.Dense(128, activation='relu'),
    layers.Dense(64, activation='relu'),
    layers.Dense(1)
])

model.compile(loss='mean_squared_error',
              optimizer=keras.optimizers.Adam(0.001),
              metrics=['mae', 'mse'])

history = model.fit(
    x=X_train, y=y_train,
    epochs=200,
    verbose=0,
    batch_size=32,
    validation_split=0.2
)

# Evaluación en conjunto de prueba
loss, mae, mse = model.evaluate(X_test, y_test, verbose=2)
print("Testing set Mean Abs Error: {:5.2f}".format(mae))

# Predicciones para el conjunto de prueba
y_pred = model.predict(X_test)

# Cálculo de métricas
mse_value = mean_squared_error(y_test, y_pred)
mae_value = mean_absolute_error(y_test, y_pred)
r2_value = r2_score(y_test, y_pred)

print(f"\nMean Squared Error (MSE): {round(mse_value, 2)}")
print(f"Mean Absolute Error (MAE): {round(mae_value, 2)}")
print(f"R2 Score: {round(r2_value, 2)}")

### **Prueba de campo con un nuevo proyecto**

In [ ]:
import pandas as pd

data = {
    'Duracion_Estimada_Meses': [9],
    'Tamaño_Equipo': [5],
    'Experiencia_Equipo_Años': [4],
    'Presupuesto_Cliente_USD': [90000],
    'Angular': [0],
    'Flutter': [0],
    'Java': [0],
    'Node.js': [1],
    'PHP': [0],
    'Python': [1],
    'React': [1],
    'Ruby on Rails': [0],
    'Swift': [1],
    'Vue.js': [0],
    'Nivel_Complejidad_Bajo': [False],
    'Nivel_Complejidad_Medio': [False],
    'Tipo_Proyecto_Desarrollo Web Corporativo': [True],
    'Tipo_Proyecto_E-commerce Nacional': [False],
    'Tipo_Proyecto_Plataforma de Educación en Línea': [False],
    'Tipo_Proyecto_Sistema ERP Local': [False]
}

dato_ejemplo = pd.DataFrame(data)

# Escalar los datos del nuevo proyecto
nuevo_proyecto_escalado = sc.transform(dato_ejemplo)

# Predecir el cobro del proyecto con cada modelo
prediccion_linear = linear_model.predict(nuevo_proyecto_escalado)[0]
prediccion_forest = forest_model.predict(nuevo_proyecto_escalado)[0]
prediccion_gradient_boosting = gradient_boosting_model.predict(nuevo_proyecto_escalado)[0]
prediccion_tensorflow = model.predict(nuevo_proyecto_escalado)[0][0]

print("\nPredicción con Regresión Lineal:", round(prediccion_linear, 2))
print("Predicción con Random Forest:", round(prediccion_forest, 2))
print("Predicción con Gradient Boosting:", round(prediccion_gradient_boosting, 2))
print("Predicción con Tensorflow:", round(prediccion_tensorflow, 2))